## 라이브러리 import

In [ ]:
import pandas            as pd
import numpy             as np
import seaborn           as sb
import scipy.stats       as sp
import matplotlib.pyplot as plt

## 데이터 불러오기

In [ ]:
FeatureData = pd.read_csv('./SavedData/FeatureData', sep=',', header=None)
FeatureData.shape

In [ ]:
# 특징값 정상, 고장 분리
Normal_Feature   = FeatureData.iloc[:,:100]
Abnormal_Feature = FeatureData.iloc[:,100:]

Normal_Feature.shape

## P-value 계산

In [ ]:
NoOfFeature = FeatureData.shape[0] # 추출된 Feature 갯수

P_value = np.zeros(shape=(NoOfFeature , 2))

# 390개 특징값 각각 T-검정 수행
for i in np.arange(NoOfFeature):
    
    T_test       = np.array(sp.ttest_ind(Normal_Feature.iloc[i,:] , Abnormal_Feature.iloc[i,:]))
    P_value[i,0] = i          # Feature Index
    P_value[i,1] = T_test[1]  # P값
    
P_value      = pd.DataFrame(P_value)
P_value_Rank = P_value.sort_values([1],ascending=True)  # P-value 기준 오름차순 정렬

P_value_Rank

In [ ]:
# 특징값 저장
path1 = './SavedData//P_value'        # path = '파일 경로//저장할 파일 이름'
path2 = './SavedData//P_value_Rank'   # path = '파일 경로//저장할 파일 이름'
P_value.to_csv(path1, sep=',', header=None, index=None)
P_value_Rank.to_csv(path2, sep=',', header=None, index=None)

## PDF(Probabilistic Density Function) 그래프 확인 (정상 vs 고장)

In [ ]:
# P-value 낮은 순위
FeatureRank = 1

# PDF 그래프
sb.distplot(Normal_Feature.iloc[int(P_value_Rank.iloc[FeatureRank-1,0])  ,:], label = 'Normal')
sb.distplot(Abnormal_Feature.iloc[int(P_value_Rank.iloc[FeatureRank-1,0]),:], label = 'Abnormal')
plt.legend(loc='upper right')
plt.show()

## P-value 기반 구분성(유의수준↓) 상위 특징값 선택

In [ ]:
Rank = 30

Normal   = np.zeros(shape=(Rank,100))
Abnormal = np.zeros(shape=(Rank,100))

for i in range(Rank):
    
    index         = int(P_value_Rank.iloc[i,0])
    Normal[i,:]   = Normal_Feature.iloc[index,:].values
    Abnormal[i,:] = Abnormal_Feature.iloc[index,:].values

# 정상, 고장 특징값 합치기    
FeatureSelected = pd.DataFrame(np.concatenate([Normal, Abnormal],axis=0))

In [ ]:
# Selected Feature 데이터 저장
path = './SavedData//FeatureSelected'   # path = '파일 경로//저장할 파일 이름'
FeatureSelected.to_csv(path, sep=',', header=None, index=None)